# Notebook 021: Baseline Classifier Model

This notebook contains a baseline crime-type classifier model using an initial set of property-related predictors

In [32]:
import urllib
import os
import operator
import pathlib
import requests
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LassoCV

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import statsmodels.api as sm

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
DATA_ROOT = '../data/'
MODEL_ROOT = '../models'
FIGURES_ROOT = '../figures/model-baseline'
WRITEDIR_ROOT = '../models/baseline'

READDIR_ROOT = os.path.join(DATA_ROOT, 'processed')
SHAPEDIR_ROOT = os.path.join(DATA_ROOT, 'raw')
FEATURES_ROOT = os.path.join(DATA_ROOT, 'processed')

readfile_model = os.path.join(READDIR_ROOT, 'crime-model-data-v1.csv')

readfile_zipshapes = os.path.join(SHAPEDIR_ROOT, 'shapefile/zipcodes/ZIP_Codes.shp')
readfile_cityshape = os.path.join(SHAPEDIR_ROOT, 'shapefile/city-boundary/City_of_Boston_Boundary.shp')
readfile_streetshapes = os.path.join(SHAPEDIR_ROOT, 'shapefile/street-segments/Boston_Street_Segments.shp')
readfile_tractshapes = os.path.join(SHAPEDIR_ROOT, 'shapefile/census-tracts/Census_2010_Tracts.shp')
readfile_hoodshapes = os.path.join(SHAPEDIR_ROOT, 'shapefile/boston-neighborhoods/Boston_Neighborhoods.shp')
readfile_zonesubshapes = os.path.join(SHAPEDIR_ROOT, 'shapefile/zoning-subdistricts/Zoning_Subdistricts.shp')
readfile_openshapes = os.path.join(SHAPEDIR_ROOT, 'shapefile/open-spaces/Open_Space.shp')

readfile_feat_property = os.path.join(FEATURES_ROOT, 'property-assessment-features-2013-2019.csv')

print(
    'readfile paths for datasets used in this notebook are:\n\t{}\n\t{}\n\t{}\n\t{}'.format(
        readfile_model, readfile_feat_property, readfile_tractshapes, WRITEDIR_ROOT
    )
)

readfile paths for datasets used in this notebook are:
	../data/processed/crime-model-data-v1.csv
	../data/processed/property-assessment-features-2013-2019.csv
	../data/raw/shapefile/census-tracts/Census_2010_Tracts.shp
	../models/baseline


In [3]:
# mkdir for saving figures if it doesn't already exist
if not os.path.exists(FIGURES_ROOT):
    os.mkdir(FIGURES_ROOT)
    
# mkdir for saving figures if it doesn't already exist
if not os.path.exists(WRITEDIR_ROOT):
    os.mkdir(WRITEDIR_ROOT)

In [4]:
df = pd.read_csv(readfile_model)

In [5]:
dropna_cols = [
    'commercial-mix-ratio',
    'industrial-mix-ratio',
    'owner-occupied-ratio',
    'residential-median-value',
    'residential-gini-coef',
    'industrial-mix-ratio-3yr-cagr',
    'commercial-mix-ratio-3yr-cagr',
    'owner-occupied-ratio-3yr-cagr',
    'residential-median-value-3yr-cagr',
    'residential-gini-coef-3yr-cagr',
]

In [6]:
df = df.dropna(subset=dropna_cols)

In [7]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149203 entries, 0 to 151071
Data columns (total 29 columns):
crime-type                           149203 non-null object
INCIDENT_NUMBER                      149203 non-null object
OFFENSE_DESCRIPTION                  149203 non-null object
timestamp                            149203 non-null object
lat                                  149203 non-null float64
lon                                  149203 non-null float64
year                                 149203 non-null int64
month                                149203 non-null int64
day-of-week                          149203 non-null object
hour                                 149203 non-null int64
ZIP5                                 149177 non-null float64
ZIP5_area                            149177 non-null float64
Name                                 149135 non-null object
Neighborhood_area                    149135 non-null float64
Neighborhood_area_2                  149135 non

,crime-type,INCIDENT_NUMBER,OFFENSE_DESCRIPTION,timestamp,lat,lon,year,month,day-of-week,hour,...,commercial-mix-ratio,industrial-mix-ratio,owner-occupied-ratio,residential-median-value,residential-gini-coef,industrial-mix-ratio-3yr-cagr,commercial-mix-ratio-3yr-cagr,owner-occupied-ratio-3yr-cagr,residential-median-value-3yr-cagr,residential-gini-coef-3yr-cagr
0,fraud,I192078177,forgery / counterfeiting,2019-08-01 17:46:00,42.304922,-71.102981,2019,8,Thursday,17,...,0.000000,0.000294,0.000000,756500.0,0.000000,-0.034814,0.000000,0.000000,0.024264,0.000000
1,harassment-disturbance,I192078061,harassment,2019-06-12 21:00:00,42.355553,-71.152747,2019,6,Wednesday,21,...,0.069416,0.000000,0.550355,745950.0,0.199606,0.000000,-0.010424,-0.028224,0.068142,0.011166
2,theft,I192078038,larceny theft of mv parts & accessories,2019-03-10 08:00:00,42.345625,-71.041291,2019,3,Sunday,8,...,0.478130,0.000938,0.456287,538500.0,0.228793,-0.084828,0.010549,-0.014131,0.060340,0.018436
4,theft,I192077997,auto theft - leased/rented vehicle,2019-04-13 08:00:00,42.328564,-71.068353,2019,4,Saturday,8,...,0.375058,0.076862,0.460751,355500.0,0.172898,0.060168,-0.007779,-0.023444,0.082123,0.025972
5,theft,I192077992,auto theft - leased/rented vehicle,2019-07-16 08:00:00,42.328564,-71.068353,2019,7,Tuesday,8,...,0.375058,0.076862,0.460751,355500.0,0.172898,0.060168,-0.007779,-0.023444,0.082123,0.025972


## Preprocess observations model dataframe 

In [8]:
print(df['day-of-week'].value_counts(dropna=False))
print()
print(df['month'].value_counts(dropna=False).sort_index())
print()
print(df['crime-type'].value_counts(dropna=False).sort_index())
print()

Friday       22445
Wednesday    21864
Thursday     21671
Saturday     21402
Monday       21350
Tuesday      21211
Sunday       19260
Name: day-of-week, dtype: int64

1     13250
2     11704
3     12878
4     13195
5     14246
6     14370
7     14637
8     14823
9     10451
10    10509
11     9654
12     9486
Name: month, dtype: int64

burglary                   6583
drugs-substances          15134
fraud                     11163
harassment-disturbance    24084
other                      7385
robbery                    3949
theft                     40204
vandalism-property        15956
violence-aggression       24745
Name: crime-type, dtype: int64



In [9]:
weekdays_list = [
    'Tuesday',
    'Wednesday',
    'Thursday',
    'Friday',
    'Saturday',
    'Sunday',
]

months_dict = {
    1: 'Jan',
    2: 'Feb',
    3: 'Mar',
    4: 'Apr',
    5: 'May',
    6: 'Jun',
    7: 'Jul',
    8: 'Aug',
    9: 'Sep',
    10: 'Oct',
    11: 'Nov',
    12: 'Dec',
}

crime_type_dict = {
    'other': 0,
    'burglary': 1,
    'drugs-substances': 2,
    'fraud': 3,
    'harassment-disturbance': 4,
    'robbery': 5,
    'theft': 6,
    'vandalism-property': 7,
    'violence-aggression': 8,
}


df['month'] = df['month'].copy().map(months_dict)
df['crime-type'] = df['crime-type'].copy().map(crime_type_dict)

print(df['day-of-week'].value_counts(dropna=False))
print()
print(df['month'].value_counts(dropna=False))
print()
print(df['crime-type'].value_counts(dropna=False))
print()

Friday       22445
Wednesday    21864
Thursday     21671
Saturday     21402
Monday       21350
Tuesday      21211
Sunday       19260
Name: day-of-week, dtype: int64

Aug    14823
Jul    14637
Jun    14370
May    14246
Jan    13250
Apr    13195
Mar    12878
Feb    11704
Oct    10509
Sep    10451
Nov     9654
Dec     9486
Name: month, dtype: int64

6    40204
8    24745
4    24084
7    15956
2    15134
3    11163
0     7385
1     6583
5     3949
Name: crime-type, dtype: int64



In [10]:
months_col_order = list(months_dict.values())[1:]

In [11]:
# one-hot-encode categorical predictors
weekday_dummies_df = pd.get_dummies(df['day-of-week']).drop(columns='Monday')[weekdays_list]
month_dummies_df = pd.get_dummies(df['month']).drop(columns='Jan')[months_col_order]


# append dummy columns to df
df[weekdays_list] = weekday_dummies_df[weekdays_list]
df[months_col_order] = month_dummies_df[months_col_order]

In [12]:
df['night'] = ((df['hour'] <4) | (df['hour']>20)).values.astype(int)

In [13]:
print(df['hour'].value_counts().sort_index())
print()
print(df['night'].value_counts().sort_index())

0      8122
1      4308
2      3555
3      2060
4      1544
5      1379
6      1927
7      3308
8      5020
9      5725
10     6850
11     7237
12     8879
13     7841
14     8168
15     8045
16     9549
17    10076
18    10177
19     8850
20     7924
21     6991
22     6315
23     5353
Name: hour, dtype: int64

0    112499
1     36704
Name: night, dtype: int64


In [14]:
df.head()

,crime-type,INCIDENT_NUMBER,OFFENSE_DESCRIPTION,timestamp,lat,lon,year,month,day-of-week,hour,...,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,night
0,3,I192078177,forgery / counterfeiting,2019-08-01 17:46:00,42.304922,-71.102981,2019,Aug,Thursday,17,...,0,0,0,0,1,0,0,0,0,0
1,4,I192078061,harassment,2019-06-12 21:00:00,42.355553,-71.152747,2019,Jun,Wednesday,21,...,0,0,1,0,0,0,0,0,0,1
2,6,I192078038,larceny theft of mv parts & accessories,2019-03-10 08:00:00,42.345625,-71.041291,2019,Mar,Sunday,8,...,0,0,0,0,0,0,0,0,0,0
4,6,I192077997,auto theft - leased/rented vehicle,2019-04-13 08:00:00,42.328564,-71.068353,2019,Apr,Saturday,8,...,1,0,0,0,0,0,0,0,0,0
5,6,I192077992,auto theft - leased/rented vehicle,2019-07-16 08:00:00,42.328564,-71.068353,2019,Jul,Tuesday,8,...,0,0,0,1,0,0,0,0,0,0


## Execute train-test-split

In [15]:
np.random.seed(10)

X_train_labels, X_test_labels, y_train, y_test = train_test_split(
    df.loc[:, df.columns != 'crime-type'], 
    df['crime-type'],
    test_size=0.2,
    random_state = 109, 
    stratify = df['crime-type']
)

print(X_train_labels.shape)
print(X_test_labels.shape)
print(y_train.shape)
print(y_test.shape)

(119362, 46)
(29841, 46)
(119362,)
(29841,)


In [16]:
# specify label columns to drop before running model
drop_cols = [
    'INCIDENT_NUMBER',
    'OFFENSE_DESCRIPTION',
    'timestamp',
    'lat',
    'lon',
    'year',
    'hour',
    'month',
    'day-of-week',
    'ZIP5',
    'ZIP5_area',
    'Name',
    'Neighborhood_area',
    'Neighborhood_area_2',
    'TRACTCE10',
    'TRACTCE10_area',
    'TRACTCE10_area_2',
    'tract-match-key',
]

In [17]:
X_train = X_train_labels.drop(columns=drop_cols)
X_test = X_test_labels.drop(columns=drop_cols)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(119362, 28)
(29841, 28)
(119362,)
(29841,)


In [18]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119362 entries, 131812 to 94612
Data columns (total 28 columns):
commercial-mix-ratio                 119362 non-null float64
industrial-mix-ratio                 119362 non-null float64
owner-occupied-ratio                 119362 non-null float64
residential-median-value             119362 non-null float64
residential-gini-coef                119362 non-null float64
industrial-mix-ratio-3yr-cagr        119362 non-null float64
commercial-mix-ratio-3yr-cagr        119362 non-null float64
owner-occupied-ratio-3yr-cagr        119362 non-null float64
residential-median-value-3yr-cagr    119362 non-null float64
residential-gini-coef-3yr-cagr       119362 non-null float64
Tuesday                              119362 non-null uint8
Wednesday                            119362 non-null uint8
Thursday                             119362 non-null uint8
Friday                               119362 non-null uint8
Saturday                             11

In [19]:
%%time

solver = 'lbfgs'
C = 100000
max_iter = 1000

# fit OVR model without polynomial features
multi_class='ovr'

OVRLogModel = LogisticRegression(
    C=C,
    solver=solver,
    max_iter=max_iter,
    multi_class=multi_class,
    random_state=20
).fit(X_train, y_train)

# # fit a model again as multinomial for comparison of results
# multi_class='multinomial'

# MNLogModel = LogisticRegression(
#     C=C,
#     solver=solver,
#     max_iter=max_iter,
#     multi_class=multi_class,
#     random_state=20
# ).fit(X_train, y_train)

CPU times: user 5.18 s, sys: 112 ms, total: 5.29 s
Wall time: 1.33 s


In [20]:
OVRLogModel

LogisticRegression(C=100000, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=20, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
OVRLogModel.score(X_test, y_test)

0.26946147917295

In [22]:
%%time

# fit a model again as multinomial for comparison of results
multi_class='multinomial'

MNLogModel = LogisticRegression(
    C=C,
    solver=solver,
    max_iter=max_iter,
    multi_class=multi_class,
    random_state=20
).fit(X_train, y_train)

CPU times: user 3.32 s, sys: 43.3 ms, total: 3.36 s
Wall time: 843 ms


In [23]:
MNLogModel

LogisticRegression(C=100000, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=20, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [24]:
MNLogModel.score(X_test, y_test)

0.26946147917295

In [25]:
def standardize_features(train_df, val_df, exclude_scale_cols):
    """
    Scales val_df features based on train_df mean and std for each feature
    and returns the scaled dataframe. To specify features not to scale,
    use the exclude_scale_cols parameter
    
    :param train_df: The training data
    :param val_df: Your test/validation data
    :param exclude_scale_cols: Optional list containing names of columns we
                               do not wish to scale
    :return: a feature-scaled version of the val_df dataframe
    """
    # create list of non-binary column names for scaling
    scaled_columns = train_df.columns.difference(exclude_scale_cols)
    
    # create StandardScaler instance fitted on non-binary train data
    Scaler = StandardScaler().fit(
        train_df[scaled_columns]
    )
    
    # scale val_df and convert to dataframe with column names
    scaled_df = pd.DataFrame(
        Scaler.transform(val_df[scaled_columns]),
        columns=scaled_columns,
    )
    
    # return full dataframe containing scaled_df merged with
    # original unscaled binary columns
    return pd.concat(
        [
            val_df.drop(scaled_columns, axis=1).reset_index(drop=True),
            scaled_df
        ],
        axis=1,
    )

In [28]:
binary_cols = [
    *weekdays_list,
    *months_col_order,
    'night'
]

X_train_scaled = standardize_features(
    X_train, X_train,
    exclude_scale_cols=binary_cols)

X_test_scaled = standardize_features(
    X_train, X_test,
    exclude_scale_cols=binary_cols)

In [31]:
%%time

solver = 'lbfgs'
C = 100000
max_iter = 1000

# fit OVR model without polynomial features
multi_class='ovr'

ScaledOVRLogModel = LogisticRegression(
    C=C,
    solver=solver,
    max_iter=max_iter,
    multi_class=multi_class,
    random_state=20
).fit(X_train_scaled, y_train)

print(ScaledOVRLogModel)
print()
print(
    ScaledOVRLogModel.score(X_test_scaled, y_test)
)

LogisticRegression(C=100000, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=20, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

0.2725444857746054
CPU times: user 13.6 s, sys: 184 ms, total: 13.8 s
Wall time: 3.46 s


In [33]:
# Normalize the values contained in X_train and X_test
scaler_fit = MinMaxScaler([0,1]).fit(X_train)

X_train_norm = pd.DataFrame(scaler_fit.transform(X_train), columns=list(X_train.columns))
X_test_norm = pd.DataFrame(scaler_fit.transform(X_test), columns=list(X_test.columns))

In [34]:
%%time

solver = 'lbfgs'
C = 100000
max_iter = 1000

# fit OVR model without polynomial features
multi_class='ovr'

NormOVRLogModel = LogisticRegression(
    C=C,
    solver=solver,
    max_iter=max_iter,
    multi_class=multi_class,
    random_state=20
).fit(X_train_norm, y_train)

print(NormOVRLogModel)
print()
print(
    NormOVRLogModel.score(X_test_norm, y_test)
)

LogisticRegression(C=100000, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=20, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

0.2725444857746054
CPU times: user 33.2 s, sys: 379 ms, total: 33.6 s
Wall time: 8.45 s


In [ ]:
%%time

solver = 'saga'
C = 1
max_iter = 10000
penalty = 'l1'

# fit OVR model without L1 regularization
multi_class='ovr'

LassoOVRLogModel = LogisticRegression(
    C=C,
    solver=solver,
    max_iter=max_iter,
    multi_class=multi_class,
    random_state=20,
    penalty=penalty
).fit(X_train, y_train)

In [ ]:
LassoOVRLogModel

In [ ]:
LassoOVRLogModel.score(X_test, y_test)

In [ ]:
%%time
# report processing time for reference

# set parameters for logistic regression with cross-validation
# I have choosen l1 penaly because it gives me far better decision boundaries
solver = 'saga'
cv = 5
max_iter = 10000
penalty = 'l1'

# fit CV OVR model with L1
multi_class='ovr'

CVOVRLogModel = LogisticRegressionCV(
    cv=cv,
    solver=solver,
    max_iter=max_iter,
    penalty=penalty,
    multi_class=multi_class,
    random_state=20
).fit(X_train, y_train)


# print resulting model object for confirmation
print("OVR model:\n{}\n".format(CVOVRLogModel))

In [ ]:
# fit a model again as multinomial for comparison of results
multi_class='multinomial'

CVMCLogModel = LogisticRegressionCV(
    cv=cv,
    solver=solver,
    max_iter=max_iter,
    penalty=penalty,
    multi_class=multi_class,
    random_state=20
).fit(X_mc_train[model_predictors], y_mc_train)

# print resulting model object for confirmation
print("Multinomial model:\n{}\n".format(CVMCLogModel))


In [ ]:
# read in required data and related shapefiles
# df_crime = pd.read_csv(readfile_crime, dtype=str)
# df_offense_mapkey = pd.read_csv(readfile_crime_match_key)
# df_offense_mapkey_2 = pd.read_csv(readfile_crime_match_key_2)
# df_sam = pd.read_csv(readfile_sam, dtype=str)
# gdf_zips = gpd.read_file(readfile_zipshapes)
# gdf_boston = gpd.read_file(readfile_cityshape)
# gdf_streets = gpd.read_file(readfile_streetshapes)
# gdf_tracts = gpd.read_file(readfile_tractshapes)
# gdf_hoods = gpd.read_file(readfile_hoodshapes)
# gdf_zonesubs = gpd.read_file(readfile_zonesubshapes)
# gdf_openspace = gpd.read_file(readfile_openshapes)

# df_feat_prop = pd.read_csv(readfile_feat_property, dtype={'shape-id': str, 'fiscal-year': int})

# # convert SAM data lat/lon values to floats
# df_sam[['X', 'Y']] = df_sam[['X', 'Y']].astype(float)
# # convert crime data lat/lon values to floats
# df_crime[['Lat', 'Long']] = df_crime[['Lat', 'Long']].astype(float)